In [4]:
import requests,re
from bs4 import BeautifulSoup
import json
import time

header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}

def page_load(url):
    results = requests.get(url,headers=header,verify=True)
    return json.loads(results.text)


class FB_crawler(object):
    def __init__(self, page_id, since, until, token, outfile, sleep=True):
        self.page_id = page_id if type(page_id) == str else str(page_id)
        self.since = since
        self.until = until
        self.token = token
        self.outfile = outfile
        self.sleep = sleep
        self.post_ids = []
        self.url_num = None
        self.data = dict()
        print("Facebook_crawler is activated...")


    def pid_collector(self):
        query = '%s/posts?fields=name,link,message&limit=100&since=%s&until=%s' % (self.page_id,self.since,self.until)
        posts = page_load("https://graph.facebook.com/v2.8/%s&access_token=%s"%(query, self.token))
        while len(posts['data']) > 0:
            self.post_ids.append(list(map(lambda x:x['id'],posts['data'])))
            posts = page_load(posts['paging']['next'])
        self.post_ids = sum(self.post_ids,[])
        self.url_num = len(self.post_ids)
        print(self.url_num,"post urls are collected")

                                 
    def crawler(self):
        assert len(self.post_ids) > 0,'please run pid_collector first or reset the search period'
        self.data['info'] = {'since':self.since,'until':self.until}
        self.data['post'] = [None]*self.url_num
        for p_idx,pid in enumerate(self.post_ids):
            # since comment crawler
            if p_idx % 20 == 0: print('crawling post {}...'.format(p_idx))
            com_query = r'%s?fields=name,description,link,message,comments' % (pid)
            com = page_load("https://graph.facebook.com/v2.8/%s&access_token=%s" % (com_query, self.token))
            self.data['post'][p_idx] = com
            while True: # restart gate for comment crawler
                try:
                    while 'paging' in com and 'next' in com['paging']:
                        com = page_load(com['paging']['next'])
                        self.data['post'][p_idx]['comments']['data'].append(com['data'])
                except:
                    print('retry')
                    time.sleep(5)
                    pass
                else:
                    break
            # start reaction crawler
            tmp = dict()
            for idx,rxn_type in enumerate(['ANGRY','SAD','LOVE','HAHA','WOW','LIKE']):
                rxn_query = '%s?fields=reactions.type(%s).summary(true)' % (pid,rxn_type)
                rxn = page_load("https://graph.facebook.com/v2.8/%s&access_token=%s" % (rxn_query, self.token)) 
                try:
                    tmp[rxn_type] = rxn['reactions']['summary']['total_count']
                except KeyError:
                    tmp[rxn_type] = None
            self.data['post'][p_idx]['reaction'] = tmp
            if p_idx % 100 == 0:
                time.sleep(5)
            json.dump(self.data, open(self.outfile, 'w', encoding='utf-8'))
        print('process complete.')

In [6]:
token='EAACEdEose0cBAB0ipsWWhrVO0pMYdvfDjapJOpYqqZAZCe7Ts7hqRw2NqputYNZCA0eF95yZAd4vb50nOX8GsBW6p9wiONZCLBR2X10eAcFV1qJHCgXydeccnXoGJ7tG5ZA0owy1VXZAcS70x1NjPg87mZCTpKurAdcTt6sS3QDZBTLZAxczhhJbU3'
c = FB_crawler('232633627068','17-03-30','17-03-31',token,'result.json')
c.pid_collector()
c.crawler()

Facebook_crawler is activated...
116 post urls are collected
crawling post 0...
crawling post 20...


KeyboardInterrupt: 

In [13]:
print(c.data['post'][55]["comments"]['data'][1])

{'message': '馬前總統嫁女兒也低調沒報、幹嘛特意要報蔡總統姪女出嫁？管很寬哦？！', 'created_time': '2017-01-01T08:30:14+0000', 'from': {'name': 'Bell Pan', 'id': '799692476717406'}, 'id': '1249935678408100_1249953408406327'}
